<a href="https://colab.research.google.com/github/Zardian18/Cybersec_assistant/blob/master/CyberAssist2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 2.2 MB/s eta 0:00:00


In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

dataset = load_dataset("ahmed000000000/cybersec")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['INSTRUCTION', 'RESPONSE'],
        num_rows: 12408
    })
})

In [6]:
import pandas as pd

In [7]:
train_ds = pd.DataFrame(dataset["train"])
train_ds

,INSTRUCTION,RESPONSE
0,Can you explain how to use WNetAddConnection t...,To use WNetAddConnection to map a network shar...
1,Compare and contrast the security features of ...,All three web servers have different security ...
2,Define 'Clickjacking' and describe how it can ...,Clickjacking is a type of attack where a malic...
3,Define 'Computed IOCs' and give examples of ea...,Computed IOCs refer to Indicators of Compromis...
4,Define 'Containment and intelligence developme...,Containment and intelligence development is a ...
...,...,...
12403,What design issue can be exploited to imperson...,The design issue that can be exploited to impe...
12404,Explain how responder-like activity can be det...,Responder-like activity in a network can be de...
12405,What is osquery and how can it be used to dete...,osquery is a tool that can be used to detect W...
12406,Explain how to detect the Netsh persistence te...,"To detect the Netsh persistence technique, fil..."


In [8]:
text_data= train_ds["RESPONSE"]
text_data

0        To use WNetAddConnection to map a network shar...
1        All three web servers have different security ...
2        Clickjacking is a type of attack where a malic...
3        Computed IOCs refer to Indicators of Compromis...
4        Containment and intelligence development is a ...
                               ...                        
12403    The design issue that can be exploited to impe...
12404    Responder-like activity in a network can be de...
12405    osquery is a tool that can be used to detect W...
12406    To detect the Netsh persistence technique, fil...
12407    The Windows event ID that is logged when an ad...
Name: RESPONSE, Length: 12408, dtype: object

In [9]:
text_data.to_csv('text.txt', index=False, header=False)

In [10]:
text_data = text_data.tolist()
text_data[:5]

['To use WNetAddConnection to map a network share, first make sure that the computer has access to the network share and that the appropriate network resource is available. Then, use the WNetAddConnection function to establish a connection to the network resource by specifying the path to the share as well as the user name and password necessary to access the share. Once connected, you can then access files and resources on the network share as if they were local to the machine. Note that the WNetAddConnection function should only be used within trusted environments, as using it to establish connections to untrusted or malicious network resources can pose a serious security threat to the system.',
 'All three web servers have different security features that set them apart. Apache is known for its mod_security module, which provides advanced security features like HTTP filtering and Real-Time Block Lists to protect against SQL injection, cross-site scripting, and other attacks. IIS has

In [11]:
!mkdir output

## Training

In [16]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Set the path to your custom dataset
text_data_path = "/content/text.txt"

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"  # You can use other variants like "gpt2-medium" or "gpt2-large"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Load your custom dataset
with open(text_data_path, "r", encoding="utf-8") as file:
    text_data = file.read()

# Tokenize the dataset
tokenized_data = tokenizer.encode(text_data, return_tensors="pt")

# Create a TextDataset and DataCollatorForLanguageModeling
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=text_data_path,
    block_size=128,  # Adjust the block size as needed
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Set to True if your dataset includes masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/output",
    overwrite_output_dir=True,
    num_train_epochs=10,  # Adjust the number of epochs as needed
    per_device_train_batch_size=16,  # Adjust batch size as needed
    save_steps=5000,  # Save model checkpoints every N steps
    save_total_limit=3,
)

# Create Trainer and fine-tune the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Token indices sequence length is longer than the specified maximum sequence length for this model (1527802 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, s

In [13]:
# !pip uninstall transformers
# !pip uninstall accelerate

In [14]:
from huggingface_hub import notebook_login
notebook_login()

In [15]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [17]:
trainer.train()

Step,Training Loss
500,2.644400
1000,2.377600
1500,2.256700
2000,2.138800
2500,2.060800
3000,2.025500
3500,1.945800
4000,1.907900
4500,1.882200
5000,1.831500


TrainOutput(global_step=7460, training_loss=1.9996440979494805, metrics={'train_runtime': 4320.4175, 'train_samples_per_second': 27.625, 'train_steps_per_second': 1.727, 'total_flos': 7796301004800000.0, 'train_loss': 1.9996440979494805, 'epoch': 10.0})

In [18]:
# trainer.push_to_hub(commit_message="Fine-tuned GPT-2 model for cybersec assistant")
model.push_to_hub("Cyber_assist2.0")
tokenizer.push_to_hub("Cyber_assist2.0")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zardian/Cyber_assist2.0/commit/9b31a30f2d09c7041f57fb4e983509ace99be1aa', commit_message='Upload tokenizer', commit_description='', oid='9b31a30f2d09c7041f57fb4e983509ace99be1aa', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# from transformers import pipeline
# model_id = "Zardian/output"
# generator = pipeline("text-generation", model=model_id)

In [19]:
import torch.nn.functional as F

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label

def sequence_logprob(model, labels, input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
            output.logits[:, :-1, :], labels[:, 1:])
        seq_log_prob = torch.sum(log_probs[:, input_len:])
    return seq_log_prob.cpu().numpy()

In [20]:
input_txt ="What is ransomware?"

In [21]:
import torch

# Check if GPU is available and choose the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
# input_txt = "Transformers are the"
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
iterations = []
n_steps = 100
choices_per_step = 5

In [23]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=150,
                               do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is ransomware? It is a type of cyber attack that involves stealing or manipulating the victim's files or data. This can be done through various means, such as exploiting vulnerabilities, stealing credit card numbers, or using social engineering techniques. The purpose of ransomware is to spread rapidly and cause significant financial losses for the victim. It is important for individuals and organizations to be aware of this threat and take proactive measures to protect their financial assets."
"Ransomware is a type of cyber attack that takes advantage of vulnerabilities in software or hardware to gain unauthorized access, control, or manipulate the victim's system. It is a type of malware that encrypts the victim's files or uses techniques to hide their presence. The purpose of ransomware is to steal money


In [24]:
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is ransomware?
Ransomware is a type of malware that encrypts files on a victim's system and demands a ransom for their release. It is commonly known as ransomware and is often used by cybercriminals to steal sensitive information and carry out malicious activities. Unlike traditional ransomware, which encrypts files on a victim's system, ransomware encrypts files on a victim's machine and then sends them to the infected system. This can be used to steal personal information, intellectual property, or even to commit financial fraud


In [25]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0]))
print(tokenizer.decode(output_greedy[0]))
print(f"\nlog-prob: {logp:.2f}")

What is ransomware? It is a type of cyber attack that involves stealing or manipulating the victim's files or data. This can be done through various means, such as exploiting vulnerabilities, stealing credit card numbers, or using social engineering techniques. The purpose of ransomware is to spread rapidly and cause significant financial losses for the victim. It is important for individuals and organizations to be aware of this threat and take proactive measures to protect their financial assets."
"Ransomware is a type of cyber attack that takes advantage of vulnerabilities in software or hardware to gain unauthorized access, control, or manipulate the victim's system. It is a type of malware that encrypts the victim's files or uses techniques to hide their presence. The purpose of ransomware is to steal money

log-prob: -134.99


In [26]:
output_beam = model.generate(input_ids, max_length=150, num_beams=5,
                             do_sample=False)
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


What is ransomware? refers to a type of cyber attack that targets computer systems, networks, or applications that are susceptible to ransomware attacks. Unlike traditional ransomware, which typically takes advantage of vulnerabilities in software or hardware, ransomware is specifically designed to evade detection by antivirus software. Unlike traditional ransomware, which typically takes advantage of vulnerabilities in software or hardware, ransomware does not encrypt or encrypt files, making it difficult to detect and remove. Unlike traditional ransomware, which typically takes advantage of vulnerabilities in software or hardware, ransomware does not encrypt or encrypt files, making it difficult to detect and remove. Unlike traditional ransomware, which typically takes advantage of vulnerabilities in software or hardware, ransomware does not encrypt or encrypt files, making it difficult to detect and remove. Unlike traditional ransomware

log-prob: -75.10


In [27]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [28]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [29]:
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.7/310.7 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.6 MB/s eta 0:00:00


In [30]:
import gradio as gr

In [31]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# def generate_text(inp):
#   input_ids = tokenizer.encode(inp, return_tensors="pt")
#   beam_output = model.generate(input_ids, max_len=150, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
#   output = tokenizer.decode(beam_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
#   return ".".join(output.split(".")[:-1])+ "."

In [32]:
def generate_text(inp):
  input_ids = tokenizer(inp, return_tensors="pt")["input_ids"].to(device)
  output = model.generate(input_ids, max_new_tokens=n_steps, do_sample=False)
  return tokenizer.decode(output[0])

In [33]:
generate_text("what is ransomware")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'what is ransomware, and what is the impact of the ransomware attack on the organization\'s cybersecurity posture."\n"The term \'Ransomware Empire\' refers to a type of ransomware that is highly sophisticated and targeted. It is known for its ability to encrypt files and spread rapidly across infected systems. Unlike traditional ransomware, which focuses on long-term storage and distribution, ransomware Empire uses a variety of techniques to deliver its malicious payloads. These techniques include exploiting vulnerabilities in software, installing backdoors, and using social engineering'

In [34]:
import gradio as gr
iface = gr.Interface(
    fn=generate_text,
    inputs="textbox",
    outputs="text",
    title="CYBERSEC_ASSISTANT",
    description="fine tuned gpt2"
)
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://24cf3a0264b265cf20.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
